In [157]:
# Install Miniconda
# https://docs.conda.io/en/latest/miniconda.html
# Python 3.7 is used

In [158]:
# Add conda channels
! conda config --add channels defaults
! conda config --add channels bioconda
! conda config --add channels conda-forge

In [159]:
# Install Gromacs
! printf "y" | conda install -c bioconda gromacs

Solving environment: done

# All requested packages already installed.



In [160]:
# Install additional libraries
! printf "y" | conda install numpy pandas matplotlib
! printf "y" | conda install nglview mdanalysis
! printf "y" | pip install GromacsWrapper pytraj

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [161]:
# Initialize and enable NGLview
! jupyter-nbextension enable --py --sys-prefix widgetsnbextension
! jupyter-nbextension enable nglview --py --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension nglview-js-widgets/extension...
      - Validating: OK


In [162]:
# numpy and matplotlib are required for result analysis and plotting

In [163]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [164]:
# pytraj and nglview are required for trajectory analysis and visualization

In [165]:
import pytraj as pt
import nglview as nv

In [166]:
# GromacsWrapper can also be used instead of command line 

In [167]:
import gromacs
import gromacs.formats
gromacs.config.setup()

In [168]:
print(gromacs.release)

2021.1-MODIFIED


In [169]:
# update the gromacs location
import os
os.environ["GMXLIB"] = "/home/" + os.environ["USER"] + "/miniconda3/envs/md/share/gromacs/top"
print(os.environ["GMXLIB"])
%env GMXLIB

/home/felix/miniconda3/envs/md/share/gromacs/top


'/home/felix/miniconda3/envs/md/share/gromacs/top'

# Generate Topology

In [170]:
# Remove water molecules
! grep -v HOH data/1aki.pdb > data/prot_clean.pdb

In [171]:
# Prepare .gro input file
! gmx pdb2gmx -f data/prot_clean.pdb -o prot_pros.gro -water spce -ff amber99sb

                 :-) GROMACS - gmx pdb2gmx, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

In [172]:
! tail topol.top

; Include topology for ions
#include "amber99sb.ff/ions.itp"

[ system ]
; Name
LYSOZYME

[ molecules ]
; Compound        #mols
Protein_chain_A     1


## Generate box and solvate protein

In [173]:
! gmx editconf -f prot_pros.gro -o prot_box.gro -c -d 1.0 -bt cubic

                :-) GROMACS - gmx editconf, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

In [174]:
! gmx solvate -cp prot_box.gro -cs spc216.gro -o prot_solv.gro -p topol.top

                 :-) GROMACS - gmx solvate, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

## Adding ions

In [175]:
! gmx grompp -f data/ions.mdp -c prot_solv.gro -p topol.top -o ions.tpr

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

In [176]:
! printf "SOL" | gmx genion -s ions.tpr -o prot_solv_ions.gro -p topol.top -pname NA -nname CL -neutral

                 :-) GROMACS - gmx genion, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

In [177]:
! tail topol.top


[ system ]
; Name
LYSOZYME in water

[ molecules ]
; Compound        #mols
Protein_chain_A     1
SOL         10636
CL               8


## Energy minimization

In [178]:
! gmx grompp -f data/minim.mdp -c prot_solv_ions.gro -p topol.top -o em.tpr

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

In [ ]:
# nt is used to define the number of cores
! gmx mdrun -v -deffnm em -nt 4

                  :-) GROMACS - gmx mdrun, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

Step=   92, Dmax= 9.6e-03 nm, Epot= -5.32612e+05 Fmax= 4.12573e+03, atom= 1015
Step=   93, Dmax= 1.1e-02 nm, Epot= -5.32934e+05 Fmax= 1.34131e+04, atom= 1015
Step=   94, Dmax= 1.4e-02 nm, Epot= -5.33332e+05 Fmax= 7.86274e+03, atom= 1015
Step=   95, Dmax= 1.7e-02 nm, Epot= -5.33378e+05 Fmax= 1.74834e+04, atom= 1015
Step=   96, Dmax= 2.0e-02 nm, Epot= -5.33815e+05 Fmax= 1.30901e+04, atom= 1015
Step=   98, Dmax= 1.2e-02 nm, Epot= -5.34213e+05 Fmax= 5.16446e+03, atom= 1015
Step=   99, Dmax= 1.4e-02 nm, Epot= -5.34360e+05 Fmax= 1.69673e+04, atom= 1015
Step=  100, Dmax= 1.7e-02 nm, Epot= -5.34850e+05 Fmax= 9.35795e+03, atom= 1015
Step=  102, Dmax= 1.0e-02 nm, Epot= -5.35125e+05 Fmax= 6.52326e+03, atom= 1015
Step=  103, Dmax= 1.2e-02 nm, Epot= -5.35315e+05 Fmax= 1.23835e+04, atom= 1015
Step=  104, Dmax= 1.5e-02 nm, Epot= -5.35602e+05 Fmax= 1.04689e+04, atom= 1015
Step=  105, Dmax= 1.8e-02 nm, Epot= -5.35662e+05 Fmax= 1.67891e+04, atom= 1015
Step=  106, Dmax= 2.1e-02 nm, Epot= -5.35928e+05 Fma

Step=  223, Dmax= 1.2e-02 nm, Epot= -5.55078e+05 Fmax= 2.97585e+03, atom= 1958
Step=  224, Dmax= 1.5e-02 nm, Epot= -5.55142e+05 Fmax= 1.94174e+04, atom= 1958
Step=  225, Dmax= 1.8e-02 nm, Epot= -5.55443e+05 Fmax= 7.80249e+03, atom= 1958
Step=  227, Dmax= 1.1e-02 nm, Epot= -5.55526e+05 Fmax= 8.42305e+03, atom= 1958
Step=  228, Dmax= 1.3e-02 nm, Epot= -5.55589e+05 Fmax= 1.12013e+04, atom= 1958
Step=  229, Dmax= 1.5e-02 nm, Epot= -5.55663e+05 Fmax= 1.21858e+04, atom= 1958
Step=  230, Dmax= 1.8e-02 nm, Epot= -5.55685e+05 Fmax= 1.60085e+04, atom= 1958
Step=  231, Dmax= 2.2e-02 nm, Epot= -5.55734e+05 Fmax= 1.76406e+04, atom= 1958
Step=  233, Dmax= 1.3e-02 nm, Epot= -5.56010e+05 Fmax= 2.63428e+03, atom= 1958
Step=  234, Dmax= 1.6e-02 nm, Epot= -5.56078e+05 Fmax= 2.13932e+04, atom= 1958
Step=  235, Dmax= 1.9e-02 nm, Epot= -5.56424e+05 Fmax= 7.88191e+03, atom= 1958
Step=  237, Dmax= 1.1e-02 nm, Epot= -5.56495e+05 Fmax= 9.56274e+03, atom= 1958
Step=  238, Dmax= 1.4e-02 nm, Epot= -5.56558e+05 Fma

Step=  354, Dmax= 1.6e-02 nm, Epot= -5.65540e+05 Fmax= 1.44843e+04, atom= 1958
Step=  355, Dmax= 1.9e-02 nm, Epot= -5.65592e+05 Fmax= 1.44124e+04, atom= 1958
Step=  357, Dmax= 1.1e-02 nm, Epot= -5.65728e+05 Fmax= 2.94643e+03, atom= 1958
Step=  358, Dmax= 1.4e-02 nm, Epot= -5.65746e+05 Fmax= 1.78346e+04, atom= 1958
Step=  359, Dmax= 1.6e-02 nm, Epot= -5.65920e+05 Fmax= 7.19514e+03, atom= 1958
Step=  361, Dmax= 9.8e-03 nm, Epot= -5.65967e+05 Fmax= 7.78616e+03, atom= 1958
Step=  362, Dmax= 1.2e-02 nm, Epot= -5.66002e+05 Fmax= 1.02382e+04, atom= 1958
Step=  363, Dmax= 1.4e-02 nm, Epot= -5.66042e+05 Fmax= 1.13400e+04, atom= 1958
Step=  364, Dmax= 1.7e-02 nm, Epot= -5.66055e+05 Fmax= 1.45812e+04, atom= 1958
Step=  365, Dmax= 2.0e-02 nm, Epot= -5.66078e+05 Fmax= 1.64564e+04, atom= 1958
Step=  367, Dmax= 1.2e-02 nm, Epot= -5.66245e+05 Fmax= 2.19977e+03, atom= 1958
Step=  368, Dmax= 1.5e-02 nm, Epot= -5.66290e+05 Fmax= 2.00885e+04, atom= 1958
Step=  369, Dmax= 1.7e-02 nm, Epot= -5.66508e+05 Fma

Step=  485, Dmax= 8.4e-03 nm, Epot= -5.72219e+05 Fmax= 7.27564e+03, atom= 1958
Step=  486, Dmax= 1.0e-02 nm, Epot= -5.72249e+05 Fmax= 8.15696e+03, atom= 1958
Step=  487, Dmax= 1.2e-02 nm, Epot= -5.72269e+05 Fmax= 1.03777e+04, atom= 1958
Step=  488, Dmax= 1.4e-02 nm, Epot= -5.72290e+05 Fmax= 1.18326e+04, atom= 1958
Step=  490, Dmax= 8.7e-03 nm, Epot= -5.72379e+05 Fmax= 1.51285e+03, atom= 1958
Step=  491, Dmax= 1.0e-02 nm, Epot= -5.72460e+05 Fmax= 1.44428e+04, atom= 1958
Step=  492, Dmax= 1.2e-02 nm, Epot= -5.72577e+05 Fmax= 4.80906e+03, atom= 1958
Step=  494, Dmax= 7.5e-03 nm, Epot= -5.72607e+05 Fmax= 6.71240e+03, atom= 1958
Step=  495, Dmax= 9.0e-03 nm, Epot= -5.72639e+05 Fmax= 7.13712e+03, atom= 1958
Step=  496, Dmax= 1.1e-02 nm, Epot= -5.72660e+05 Fmax= 9.45157e+03, atom= 1958
Step=  497, Dmax= 1.3e-02 nm, Epot= -5.72686e+05 Fmax= 1.04742e+04, atom= 1958
Step=  498, Dmax= 1.6e-02 nm, Epot= -5.72688e+05 Fmax= 1.33968e+04, atom= 1958
Step=  499, Dmax= 1.9e-02 nm, Epot= -5.72699e+05 Fma

In [ ]:
! printf "10 0" | gmx energy -f em.edr  -o potential.xvg
# 10 is the potential, maybe the number is different in a different version of gromacs

In [ ]:
! cat potential.xvg

In [ ]:
# generate graph
potential = np.genfromtxt([i for i in open('potential.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(potential)
plt.plot(*potential.T)
plt.xlabel('step')
plt.ylabel('potential')

## Equilibration Steps - NVT

In [ ]:
! gmx grompp -f data/nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
! gmx mdrun -v -deffnm nvt -nt 4

In [ ]:
! printf "16 0" | gmx energy -f nvt.edr -o temperature.xvg

In [ ]:
# generate graph
temperature = np.genfromtxt([i for i in open('temperature.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(temperature)
plt.plot(*temperature.T)
plt.xlabel('time / ps')
plt.ylabel('temperature')

In [ ]:
! gmx grompp -f data/npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
! gmx mdrun -deffnm npt -nt 4

In [ ]:
! printf "18 0" | gmx energy -f npt.edr -o pressure.xvg

In [ ]:
! printf "24 0" | gmx energy -f npt.edr -o density.xvg

In [ ]:
# generate graph
pressure = np.genfromtxt([i for i in open('pressure.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(pressure)
plt.plot(*pressure.T)
plt.xlabel('time / ps')
plt.ylabel('pressure')

In [ ]:
# generate graph
density = np.genfromtxt([i for i in open('density.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(density)
plt.plot(*density.T)
plt.xlabel('time / ps')
plt.ylabel('density')

## Production MD Run

In [ ]:
! gmx grompp -f data/md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr

In [ ]:
! gmx mdrun -deffnm md_0_1 -nt 4

## Trajectory Visualization

In [ ]:
! printf "Protein\nSystem\n" | gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -center -ur compact -pbc mol -o md_0_1_center.xtc

In [ ]:
! printf "Backbone\nSystem\n" | gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -fit rot+trans -o md_0_1_fit.xtc

In [ ]:
traj1 = pt.load(nv.datafiles.XTC, nv.datafiles.PDB)
view1 = nv.show_pytraj(traj1)
view1

In [ ]:
traj2 = pt.datafiles.load_tz2()
c = view1.add_trajectory(traj2)
view2 = nv.show_pytraj(traj2)
view2

## Analysis

In [ ]:
! printf "Protein\nSystem\n" | gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -o md_0_1_noPBC.xtc -pbc mol -center

In [ ]:
! printf "Backbone\nSystem\n" | gmx rms -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsd.xvg -tu ns

In [ ]:
# generate graph
rmsd = np.genfromtxt([i for i in open('rmsd.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(rmsd)
plt.plot(*rmsd.T)
plt.xlabel('time / ns')
plt.ylabel('rmsd')

In [ ]:
! printf "Protein\nSystem\n" | gmx gyrate -s md_0_1.tpr -f md_0_1_noPBC.xtc -o gyrate.xvg

In [ ]:
# generate graph
gyrate = np.genfromtxt([i for i in open('gyrate.xvg').read().splitlines()
                          if not i.startswith(('#', '@'))])
print(rmsd)
plt.plot(*rmsd.T)
plt.xlabel('time / ns')
plt.ylabel('gyrate')